# 1. Load Data

In [28]:
# Load data
import os
import pandas as pd
import glob
import json
import numpy as np
from tqdm.notebook import tqdm
import torch
import gc
import ipywidgets
from joblib import Parallel, delayed
import geopandas as gpd
from sklearn.neighbors import BallTree, NearestNeighbors


# Confirm the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/qusta100/STGNN


In [2]:
# Path to the price data folder
folder_path = "/gpfs/scratch/qusta100/STGNN/Data/Prices/"

# Find all CSV files matching the pattern "*-prices.csv"
csv_files = glob.glob(os.path.join(folder_path, "*-prices.csv"))

# List to store daily data
df_list = []

# Read each file and add it to the list
for file in csv_files:
    # Read: timestamp, UUID, Diesel, E5, E10
    df_day = pd.read_csv(file, usecols=[0, 1, 2, 3, 4])
    df_day.columns = ["date", "station_uuid", "diesel", "e5", "e10"]
    df_list.append(df_day)

# Combine all daily data into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

# Ensure correct column names (optional)
df.columns = ["date", "station_uuid", "diesel", "e5", "e10"]

# Parse timestamps and remove timezone
df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)

# Sort by station and time
df = df.sort_values(["station_uuid", "date"])

# Get the overall time range (start & end)
start = df["date"].min().replace(minute=0, second=0)
end = df["date"].max().replace(minute=45, second=0)

# Create 15-minute time intervals
quarter_hours = pd.date_range(start=start, end=end, freq="15min")


In [3]:
# Load all station files from March and April
station_files = glob.glob("/gpfs/scratch/qusta100/Gasoline/Data/Stations/2025-0[3-4]-*-stations.csv")

stations_list = []
for file in station_files:
    df_station = pd.read_csv(file)
    # Extract the date from the filename and convert to datetime
    date_str = os.path.basename(file).split("-stations.csv")[0]
    df_station["day"] = pd.to_datetime(date_str)
    stations_list.append(df_station)

# Combine all station data into a single DataFrame
stations_all = pd.concat(stations_list, ignore_index=True)

# Keep only the latest entry per station (based on the 'day' column)
stations_latest = stations_all.sort_values("day").drop_duplicates("uuid", keep="last")

In [4]:
# Merge price data with the latest station information
df = df.merge(
    stations_latest,
    how="left",
    left_on="station_uuid",
    right_on="uuid"
)

# 2. Sampling

In [5]:
# Clean coordinates
def to_scalar(v):
    if isinstance(v, (list, tuple, np.ndarray)):
        return v[0] if len(v) else np.nan
    return v

lon = pd.to_numeric(df["longitude"].map(to_scalar), errors="coerce")
lat = pd.to_numeric(df["latitude"].map(to_scalar),  errors="coerce")

# 2) Vectorize Coordinates
geom = gpd.points_from_xy(lon, lat)  # erwartet WGS84

gdf = gpd.GeoDataFrame(df.copy(), geometry=geom, crs="EPSG:4326")

# 3) Load shape files
nrw = gpd.read_file("/gpfs/scratch/qusta100/STGNN/Data/Shapes/thuringia.geojson").to_crs("EPSG:4326")

# 4) Spatial Join
joined = gpd.sjoin(gdf, nrw[["geometry"]], how="left", predicate="within")
gdf["in_thuringia"] = joined.index_right.notna()

ERROR 1: PROJ: proj_create_from_database: Open of /gpfs/project/qusta100/pytorch/share/proj failed


In [6]:
# 4b) Initialize in_region: 1 for Thuringia, 0 otherwise
gdf["in_region"] = gdf["in_thuringia"].fillna(False).astype(np.int8)

# 5) Find neighbors within 30 km using BallTree (memory-efficient batch version)
valid = gdf["latitude"].notna() & gdf["longitude"].notna()
th_mask = gdf["in_thuringia"].fillna(False) & valid

if th_mask.any():
    lat_v = gdf.loc[valid, "latitude"].to_numpy(dtype=np.float32)
    lon_v = gdf.loc[valid, "longitude"].to_numpy(dtype=np.float32)
    coords_rad = np.deg2rad(np.column_stack([lat_v, lon_v])).astype(np.float32, copy=False)

    # Store the DataFrame indices of valid rows (to map results back later)
    valid_idx = gdf.index[valid].to_numpy()

    # Query points: only Thuringia stations
    th_lat = gdf.loc[th_mask, "latitude"].to_numpy(dtype=np.float32)
    th_lon = gdf.loc[th_mask, "longitude"].to_numpy(dtype=np.float32)
    th_coords_rad = np.deg2rad(np.column_stack([th_lat, th_lon])).astype(np.float32, copy=False)

    # Build BallTree using Haversine distance (spherical)
    tree = BallTree(coords_rad, metric="haversine")

    # Radius in radians (30 km on Earth’s surface)
    R_EARTH_KM = np.float32(6371.0088)
    radius_rad = np.float32(30.0) / R_EARTH_KM

    # Boolean mask for all valid rows; will be True for stations in the region
    region_mask_valid = np.zeros(coords_rad.shape[0], dtype=bool)

    # Batch query
    BATCH = 5000
    for start in range(0, th_coords_rad.shape[0], BATCH):
        end = start + BATCH
        neigh_lists = tree.query_radius(th_coords_rad[start:end], r=radius_rad)

        # Instead of concatenating large arrays, mark neighbors directly in the mask
        for arr in neigh_lists:
            if arr.size:
                region_mask_valid[arr] = True

    # Ensure all Thuringia points themselves are included
    region_mask_valid[np.searchsorted(valid_idx, gdf.index[th_mask])] = True

    # Map back to global DataFrame indices and set in_region = 1
    gdf.loc[valid_idx[region_mask_valid], "in_region"] = 1

# 6) Create a plain DataFrame without geometry
ndf = pd.DataFrame(gdf).drop(columns="geometry")

In [29]:
# Number of entries in Thuringia
print(len(ndf[ndf["in_thuringia"] == True]))
print(len(ndf[ndf["in_region"] == 1]))

# Filter only entries in the wanted region
ndf = ndf[ndf["in_region"]==1]

823573
1943147


# 3. Process Data

In [17]:
# Result list
resampled_list = []

# List of fuel price columns
price_cols = ["diesel", "e5", "e10"]

# List of metadata columns (everything except date, UUID, and prices)
metadata_cols = [col for col in ndf.columns if col not in ["date", "station_uuid"] + price_cols]

# For each station: resample time series and attach metadata
for station_id, group in ndf.groupby("station_uuid"):
    group = group.set_index("date").sort_index()

    # Resample prices (forward fill) for each fuel type
    reindexed_prices = {
        col: group[col].reindex(quarter_hours, method="ffill")
        for col in price_cols
    }

    # DataFrame with timestamps and UUID
    result = pd.DataFrame({
        "date": quarter_hours,
        "station_uuid": station_id,
        **{col: reindexed_prices[col].values for col in price_cols}
    })

    # Metadata from the first row (if available)
    meta = group[metadata_cols].iloc[0] if not group.empty else pd.Series(index=metadata_cols)

    # Attach metadata to each row
    for col in metadata_cols:
        result[col] = meta[col]

    resampled_list.append(result)

# Merge all resampled data
resampled_df = pd.concat(resampled_list, ignore_index=True)

# Restrict to April
resampled_df = resampled_df[resampled_df["date"] >= pd.Timestamp("2025-04-01 00:00:00")]

In [18]:
# Remove rows with missing metadata
df_clean = resampled_df[resampled_df[["latitude", "longitude"]].notna().all(axis=1)].copy()

# Remove rows with all prices missing
df_clean = df_clean[df_clean[["diesel", "e5", "e10"]].notna().any(axis=1)].copy()

In [19]:
df_clean["date"] = pd.to_datetime(df_clean["date"])

# 1) Last observation date per station
last_seen = (
    df_clean.groupby("station_uuid", as_index=False)["date"]
    .max()
    .rename(columns={"date": "last_seen"})
)

# 2) Create a new DataFrame
df_filtered = df_clean.merge(last_seen, on="station_uuid", how="left")
df_filtered["last_seen"] = pd.to_datetime(df_filtered["last_seen"])

# 3) Moving activity and price filter
cutoff = pd.Timedelta(days=15)
mask_active = (df_filtered["date"] - df_filtered["last_seen"]) <= cutoff
mask_price  = df_filtered["diesel"] > 0

df_filtered = df_filtered[mask_active & mask_price].copy()

print("Obs after filter:", len(df_filtered))

Obs after filter: 2963412


In [20]:
# Replace 0 with NaN for all price columns
price_columns = ["diesel", "e5", "e10"]

# Replace 0 and negative values with NA in price columns (if they exist)
for col in price_columns:
    if col in df_filtered.columns:
        df_filtered[col] = df_filtered[col].apply(lambda x: pd.NA if x <= 0 else x)

In [21]:
# Handling opening times
def is_open(opening_json, timestamp):
    try:
        if opening_json.strip() == "{}":
            return True  # always open according to MTS-K

        data = json.loads(opening_json)
        weekday = timestamp.weekday()  # 0 = Monday, ..., 6 = Sunday
        time_str = timestamp.strftime("%H:%M")

        for block in data.get("openingTimes", []):
            if block.get("applicable_days", 0) & (1 << weekday):
                for period in block.get("periods", []):
                    if period["startp"] <= time_str < period["endp"]:
                        return True
        return False  # none of the time blocks apply
    except:
        return True  # if uncertain: treat as open

# Create a copy
df_clean_opening = df_filtered.copy()

# Price columns to set to NaN if station is closed
price_columns = [col for col in ["diesel", "e5", "e10"] if col in df_clean_opening.columns]

# Calculate opening status for each row
df_clean_opening["is_open"] = df_clean_opening.apply(
    lambda row: is_open(row["openingtimes_json"], row["date"]), axis=1
)

# Set all prices to NaN if closed
for col in price_columns:
    df_clean_opening.loc[~df_clean_opening["is_open"], col] = pd.NA


In [22]:
# Verification and exclusion of highway stations using the stations dataset
autobahn_pattern = (
    r"\b(?:"
    r"autobahn(?:\s*tankstelle)?"
    r"|autohof"
    r"|rasthof"
    r"|rast[-\s]?stätte"
    r"|raststaette"
    r"|tank\s*(?:und|&)\s*rast"
    r"|service\s*area"
    r"|an\s+der\s+a[0-9]{1,2}"
    r"|a\s*[0-9]{1,2}"
    r"|ausfahrt"
    r"|abfahrt"
    r"|bundesautobahn"
    r")\b"
)

# 2. Identify highway stations by name or street
is_highway = (
    stations_latest["name"].str.contains(autobahn_pattern, case=False, na=False) |
    stations_latest["street"].str.contains(autobahn_pattern, case=False, na=False)
)

# 3. Get UUIDs of identified highway stations
highway_ids = stations_latest[is_highway]["uuid"].unique()

# 4. Exclude these stations from df_clean_opening
df_clean_opening_excl_highway = df_clean_opening[
    ~df_clean_opening["station_uuid"].isin(highway_ids)
].copy()

# 5. Output
print("Number of detected highway stations:", len(highway_ids))
print("Rows before exclusion:", len(df_clean_opening))
print("Rows after exclusion:", len(df_clean_opening_excl_highway))

Number of detected highway stations: 452
Rows before exclusion: 2963412
Rows after exclusion: 2868636


# 4. Adding more features

In [23]:
# Add Features
df = df_clean_opening_excl_highway.copy()

# Assuming your DataFrame is named df and the timestamp column is 'date'
s = df['date'].astype(str) \
              .str.replace('\u00a0', ' ', regex=False) \
              .str.strip() \
              .str.replace(r'\s+', ' ', regex=True)
parsed_full  = pd.to_datetime(s, format='%Y-%m-%d %H:%M:%S', errors='coerce')
parsed_date  = pd.to_datetime(s, format='%Y-%m-%d', errors='coerce')
df['date']   = parsed_full.fillna(parsed_date)

# 1) Weekday in German (Monday–Sunday)
# Method A: using German locale (works if installed on your system)
df['weekday'] = df['date'].dt.day_name(locale='de_DE')

# Alternatively, if the locale isn't available, manual mapping:
weekday_map = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
df['weekday'] = df['date'].dt.weekday.map(weekday_map)

# 2) Holiday dummy (1 if date is 2025-04-18 or 2025-04-21, otherwise 0)
holidays = {pd.Timestamp('2025-04-18'), pd.Timestamp('2025-04-21')}
# .normalize() resets the time to midnight so you compare only dates
df['holiday'] = df['date'].dt.normalize().isin(holidays).astype(int)

# 3) Time formatted as "hh:mm"
df['time'] = df['date'].dt.strftime('%H:%M')

# Example output
print(df[['date', 'weekday', 'holiday', 'time']].head())

                 date  weekday  holiday   time
0 2025-04-01 00:00:00  Tuesday        0  00:00
1 2025-04-01 00:15:00  Tuesday        0  00:15
2 2025-04-01 00:30:00  Tuesday        0  00:30
3 2025-04-01 00:45:00  Tuesday        0  00:45
4 2025-04-01 01:00:00  Tuesday        0  01:00


In [ ]:
# Import raw oil prices via API
# Because of a missing network connection here, this step was run on a different server

import yfinance as yf
import pandas as pd

# 1. Retrieve hourly Brent data from April 1 to April 30, 2025
ticker = yf.Ticker("BZ=F")
df = ticker.history(start="2025-04-01", end="2025-05-01", interval="60m")
df = df.reset_index()
df['Datetime'] = df['Datetime'].dt.tz_localize(None)

# 2. Generate a complete 15-minute timestamp grid for April 2025
all_times = pd.date_range(
    start="2025-04-01 00:00:00",
    end="2025-04-30 23:45:00",
    freq="15min"
)
full_df = pd.DataFrame({'Datetime': all_times})

# 3. Select only the 'Open' price and rename it to 'Brent_Price'
df_short = df[['Datetime', 'Open']].rename(columns={'Open': 'Brent_Price'})

# 4. Merge the full 15-minute grid with the hourly prices,
#    filling each 15-min slot with the most recent (backward fill) hourly price
merged = pd.merge_asof(
    full_df.sort_values('Datetime'),
    df_short.sort_values('Datetime'),
    on='Datetime',
    direction='backward'
)

# 5. Save the result to a CSV file
merged.to_csv("brent_april_2025_15min_filled.csv", index=False)

# Optional: display the first 20 rows and total count (should be 2880 timestamps)
print(merged.head(20))
print("Total number of timestamps:", len(merged))

In [25]:
# Load data (restore from saved file)
brent = pd.read_csv("/gpfs/scratch/qusta100/STGNN/Data/Temp/brent_april_2025_15min_filled.csv")
brent = brent.rename(columns={"Datetime": "date"})
brent['date'] = pd.to_datetime(brent['date'])

In [39]:
# Merge with main data set (df_main)
final_df = pd.merge(df, brent, on='date', how='left')

# Show data set
final_df.head()

,date,station_uuid,diesel,e5,e10,uuid,name,brand,street,house_number,...,openingtimes_json,day,in_thuringia,in_region,last_seen,is_open,weekday,holiday,time,Brent_Price
0,2025-04-01 00:00:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,{},2025-04-30,False,1,2025-04-30 21:45:00,True,Tuesday,0,00:00,74.959999
1,2025-04-01 00:15:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,{},2025-04-30,False,1,2025-04-30 21:45:00,True,Tuesday,0,00:15,74.959999
2,2025-04-01 00:30:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,{},2025-04-30,False,1,2025-04-30 21:45:00,True,Tuesday,0,00:30,74.959999
3,2025-04-01 00:45:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,{},2025-04-30,False,1,2025-04-30 21:45:00,True,Tuesday,0,00:45,74.959999
4,2025-04-01 01:00:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,{},2025-04-30,False,1,2025-04-30 21:45:00,True,Tuesday,0,01:00,74.959999


In [27]:
# Save the final dataframe to CSV
final_df.to_csv("/gpfs/scratch/qusta100/STGNN/Data/Temp/final.csv", index=False)